In [1]:
%%capture
import sys

# # If you're on Colab:
# if 'google.colab' in sys.modules:
#     DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
#     !pip install category_encoders==2.*

# # If you're working locally:
# else:
DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
!pip install pandas-profiling==2.7.1
!pip install category_encoders==2.*

In [2]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [3]:
import numpy as np

# Wrangle the modeling data

# indicate_missing is a function that returns a boolean value if the inbound data = 'MISSING'
#   - helps in creating a "missing" column 
def indicate_missing(val):
  if val == 'MISSING':
    return True

  return False

# boolean_missing is a function converting the permit boolean column to categorical data 
def boolean_missing(val):
  if val == True:
    return 'TRUE'

  if val == False:
    return 'FALSE'

  return 'MISSING'

def wrangle(DF):
  X = DF.copy()

  # Replace near zero latitude values with zero
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  # Replace zero values with nan so we can impute values downstream
  cols_with_zeroes = ['longitude',
                      'latitude',
                      'construction_year',
                      'gps_height',
                      'population']
  for col in cols_with_zeroes:
    X[col] = X[col].replace(0, np.nan)   # replace zeros with nans
    X[col+'_MISSING'] = X[col].isnull()  # create a missing indicator column

  # Create columns for month and year recorded data
  X['date_recorded']  = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
  X['year_recorded']  = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month

  # Create a column reflecting the number of years from construction to year recorded
  X['years']          = X['year_recorded'] - X['construction_year']
  X['years']          = X['years'].replace(0, np.nan)   # replace zeros with nans
  X['years_MISSING']  = X['years'].isnull() # create a missing years indicator column

  # Replace missing boolean data with categorical data reflecting that missing data
  cols_boolean_missing = ['public_meeting', 'permit']
  for col in cols_boolean_missing:
    X[col+'_CATEGORICAL']             = X[col].apply(boolean_missing)
    X[col+'_CATEGORICAL'+'_MISSING']  = X[col+'_CATEGORICAL'].apply(indicate_missing)

  # Replace missing categorical data with 'MISSING'
  cols_categorical_missing = ['funder', 'installer', 'scheme_name', 'scheme_management', 'subvillage']
  for col in cols_categorical_missing:
    X[col]            = X[col].replace(np.nan, 'MISSING')   # replace zeros with nans
    X[col+'_MISSING'] = X[col].apply(indicate_missing)

  # List columns to be dropped
  cols_drop = [
               'date_recorded',             # date_recorded - using year_recorded and month_recorded instead
               'quantity_group',            # duplicate column
               'payment_type',              # duplicate column
               'recorded_by',               # data collection process column (not predictive)
               'id',                        # data collection process column (not predictive)
               'permit',                    # replaced by categorical column: permit_string
               'num_private',               # 98% zeroes, unclear the purpose of this dat
               'construction_year',         # use 'years' as a proxy
               'construction_year_MISSING', # use 'years_MISSING' as a proxy
               'amount_tsh']                # highly skewed data
  # Also drop the columns we processed due to missing values
  cols_drop.extend(cols_boolean_missing)

  # Drop undesired columns
  X = X.drop(columns=cols_drop)

  return X

In [4]:
# initial wrangle for train and test data
df_train = wrangle(train)
df_test  = wrangle(test)

print(f'Train shape: {df_train.shape} Test shape: {df_test.shape}')

Train shape: (59400, 48) Test shape: (14358, 47)


In [5]:
# Import pandas profiling package
import pandas_profiling
from pandas_profiling import ProfileReport

# Generate and download profile reports for the wrangled train and test datasets
ProfileReport(df_train, minimal=True).to_file(output_file="tanzania_train_output.html")
ProfileReport(df_test,  minimal=True).to_file(output_file="tanzania_test_output.html")

In [6]:
# The status_group column is the target
target = 'status_group'

# Define the training model inputs and target columns
features = df_train.columns.drop(target)
X_train  = df_train[features]
y_train  = df_train[target]

In [7]:
y_train.sample(15)

28445                 functional
43507                 functional
28560             non functional
39597                 functional
8833                  functional
36335                 functional
57510             non functional
35478             non functional
20763             non functional
44373                 functional
16722    functional needs repair
31954                 functional
14511    functional needs repair
33117             non functional
22328                 functional
Name: status_group, dtype: object

In [8]:
import category_encoders as ce 
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

# Construct the Cross Validation pipeline and fold configuration
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(random_state=42)
)

In [9]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Configure ranges of hyperparameter distributions
param_dist = {
    'randomforestclassifier__n_estimators': randint(50, 500),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],
    'randomforestclassifier__max_features': uniform(0, 1)
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    scoring=None,
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

In [10]:
y_train.sample(15)

58227        functional
8621         functional
54240    non functional
40185    non functional
51001    non functional
26420        functional
58139        functional
51220    non functional
5728         functional
11243        functional
37570        functional
49809        functional
7512         functional
44042        functional
50874        functional
Name: status_group, dtype: object

In [11]:
search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done  11 out of  30 | elapsed:  2.5min remaining:  4.3min
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed:  3.5min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:  4.5min remaining:  2.6min
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:  4.9min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:  5.1min remaining:   33.7s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  5.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,


In [12]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.7946634753939757, 'randomforestclassifier__n_estimators': 369}
Cross-validation MAE -0.8017845117845118


In [13]:
# Generate predictions
pipeline_best = search.best_estimator_

X_test = df_test[features]

# Generate model 'status_group' predictions for test dataset
y_pred = pipeline_best.predict(X_test)